In [1]:
#Importing Python Libaries

import psycopg2 as pg
import pandas as pd 
import xlsxwriter
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase 
from email import encoders 
import datetime
import os

os.remove(r'C://Users//sachi//OneDrive//Desktop/WMS_Stock_Level.xlsx')

#Connecting to the database

conn = pg.connect( user = 'doadmin', 
                   password = 'xpmt05ij9uf9rknn', 
                   host = 'tjori-bi-do-user-6486966-0.db.ondigitalocean.com', 
                   port = '25060', 
                   database = 'defaultdb')

#Defining a function, in which quering the data using SQL

def wms_stock_level():
    sql = '''SELECT 
                b.category AS category
               ,b.sku AS product_sku
               ,b.size AS product_size
               ,COUNT(a.in_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata') AS inward_stock
               ,COUNT(a.out_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata') AS outward_stock
               ,COUNT(a.in_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata') - COUNT(a.out_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata') AS current_stock_level
               ,coalesce(sp.special_price,0) AS special_price
               ,((COUNT(a.in_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata') - COUNT(a.out_timestamp::TIMESTAMP AT TIME ZONE 'utc' AT TIME ZONE 'asia/kolkata')) * (coalesce(sp.special_price,0))) AS current_stock_valuation
    
            FROM 
               inventory_skubin a
               LEFT JOIN inventory_skuean b ON a.sku_id = b.id
               LEFT JOIN store_product AS sp ON b.sku = sp.sku

           GROUP BY
               b.category
              ,b.sku
              ,b.size
              ,sp.special_price

          ORDER BY
              current_stock_level DESC

;		
'''
    return pd.read_sql_query(sql,conn)


#Calling a function, and store the query output in a temporary variable

df = wms_stock_level()

def write_to_excel():
    writer = pd.ExcelWriter('C://Users//sachi//OneDrive//Desktop//WMS_Stock_Level.xlsx', engine='xlsxwriter')
    df.to_excel(writer,sheet_name='Current_Stock_Level') 
    writer.save()
    
def send_email():
    sender = "nimit@tjori.com"
    recievers = ["nimit@tjori.com",
                 "ankit@tjori.com",
                 "mansi@tjori.com",
                 "mkindra@tjori.com",
                 "poonam@tjori.com",
                 "pavitra@tjori.com",
                 "shubhangi@tjori.com",
                 "shruti@tjori.com",
                 "sabhyata@tjori.com",
                 "akanksha@tjori.com",
                 "geetika@tjori.com",
                 "nikita@tjori.com",
                 "ritika.chhokra@tjori.com"
                ]

    # Create message container 
    
    msg = MIMEMultipart('alternative')
    msg['Subject'] = "Daily Warehouse Stock List"
    msg['From'] = sender
    msg['To'] = ",".join(recievers)

    # instance of MIMEBase and named as part
    
    part = MIMEBase('application', "octet-stream")
    part.set_payload(open("C:/Users/sachi/OneDrive/Desktop/WMS_Stock_Level.xlsx", "rb").read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', 'attachment; filename="WMS_Stock_Level.xlsx"')
    msg.attach(part)

    # creates SMTP session 
    
    mail = smtplib.SMTP('smtp.gmail.com', 587)
    mail.ehlo()
    mail.starttls()
    mail.login('nimit@tjori.com', 's/987456321/G')
    mail.sendmail(sender, recievers, msg.as_string())
    mail.quit()
    

def main():
    while True:
        write_to_excel()
        send_email()
        break
        
if __name__ == '__main__':
    main()
        


    

